<a href="https://colab.research.google.com/github/awaisshafi6164/text_summarizer_AI_model/blob/main/text_summarizer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
trainning_df = pd.read_csv('/content/train.csv')

In [3]:
MAX_LEN = 500
SUMMARY_LEN = 150
TRAINNING_SIZE = 2000

In [4]:
trainning_df = trainning_df.iloc[0:TRAINNING_SIZE,:].copy()

trainning_article_ls = list(trainning_df['Abstract'])
trainning_highlight_ls = list(trainning_df['RHS'])

del trainning_df

In [5]:
from random import sample

for index in sample(list(np.arange(len(trainning_article_ls))),2):
    print('Original Text : ')
    print(trainning_article_ls[index])

    print('\n\nSummary Text : ')
    print(trainning_highlight_ls[index])

    print('===========================================================================================================\n\n')

Original Text : 
 Cell counting is one of the basic needs of most biological experiments . Numerous methods and systems have been studied to improve the reliability of counting . However at present manual cell counting performed with a hemocytometer still represents the gold standard despite several problems limiting reproducibility and repeatability of the counts and at the end jeopardizing their reliability in general . We present our own approach based on image processing techniques to improve counting reliability . It works in two stages first building a high resolution image of the hemocytometer s grid then counting the live and dead cells by tagging the image with flags of different colours . In particular we introduce GridMos a fully automated mosaicing method to obtain a mosaic representing the whole hemocytometer s grid . In addition to offering more significant statistics the mosaic freezes the culture status thus permitting analysis by more than one operator . Finally the mo

In [6]:
print(len(trainning_article_ls))
print(len(trainning_highlight_ls))

2000
2000


In [7]:
df = pd.DataFrame(columns=['target_text','source_text'])
df['target_text'] = trainning_highlight_ls
df['source_text'] = [item for item in trainning_article_ls]

In [8]:
df.head()

,target_text,source_text
0,We generalize polar harmonic transforms for pa...,This paper introduces four classes of rotatio...
1,Design of a MATLAB based GUI tool for general ...,In this paper a MATLAB based graphical user i...
2,A markerless low cost system for the estimatio...,Background and objective A markerless low cos...
3,We adopt Distribution Field DF layer as featur...,This paper presents an improved multiple inst...
4,A system that constructs multi instance bags f...,Text based image retrieval may perform poorly...


In [9]:
!pip install setuptools>=60.0.0

In [10]:
!pip install simplet5 -q

In [11]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


In [12]:
MAX_EPOCHS = 5

In [13]:
model.train(train_df=df[0:(int)(0.7*TRAINNING_SIZE)],
            eval_df=df[(int)(0.7*TRAINNING_SIZE):TRAINNING_SIZE],
            source_max_token_len=MAX_LEN,
            target_max_token_len=SUMMARY_LEN,
            batch_size=8, max_epochs=MAX_EPOCHS, use_gpu=True)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [14]:
import os
model_path = ''
rootdir = 'outputs/'
for it in os.scandir(rootdir):
    if it.is_dir():
        if 'simplet5-epoch-'+(str)(MAX_EPOCHS-1) in it.path:
            model_path = it.path
            print(model_path)

outputs/simplet5-epoch-4-train-loss-1.9127-val-loss-2.3496


In [15]:
model.load_model("t5","./"+model_path, use_gpu=True)

In [16]:
import torch
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

no_tune_model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device('cpu')

In [17]:
from random import sample

for index in sample(list(np.arange(len(trainning_article_ls))),3):
    print('Original Text : ')
    print(trainning_article_ls[index])

    print('\n\nSummary Text : ')
    print(trainning_highlight_ls[index])

    print('\n\nFine tuned Predicted Summary Text : ')
    print(model.predict(trainning_article_ls[index]))

    print('\n\nNot Fine tuned Predicted Summary Text : ')
    preprocess_text = trainning_article_ls[index].strip().replace("\n","")
    t5_prepared_Text = "summarize: "+preprocess_text
    tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
    summary_ids = no_tune_model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=30,
                                        max_length=SUMMARY_LEN,
                                        early_stopping=True)
    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(output)
    print('===========================================================================================================\n\n')

Original Text : 
 Twitter is a popular social network which allows millions of users to share their opinions on what happens all over the world . In this work we present a system for real time Twitter data analysis in order to follow popular events from the user s perspective . The method we propose extends and improves the Soft Frequent Pattern Mining algorithm by overcoming its limitations in dealing with dynamic real time detection scenarios . In particular in order to obtain timely results the stream of tweets is organized in dynamic windows whose size depends both on the volume of tweets and time . Since we aim to highlight the user s point of view the set of keywords used to query Twitter is progressively refined to include new relevant terms which reflect the emergence of new subtopics or new trends in the main topic . The real time detection system has been evaluated during the 2014 FIFA World Cup and experimental results show the effectiveness of our solution . 



Summary Tex

In [18]:
df = pd.read_csv('/content/val.csv')

for index in sample(list(np.arange(len(df))),3):
    print('Original Text : ')
    print(df['RHS'][index])

    print('\n\nFine tuned Predicted Summary Text : ')
    print(model.predict(df['RHS'][index]))

    print('\n\nNot Fine tuned Predicted Summary Text : ')
    preprocess_text = df['RHS'][index].strip().replace("\n","")
    t5_prepared_Text = "summarize: "+preprocess_text
    tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)
    summary_ids = no_tune_model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=30,
                                        max_length=SUMMARY_LEN,
                                        early_stopping=True)
    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(output)
    print('===========================================================================================================\n\n')

Original Text : 
Solid shell elements attending shear trapezoidal and thickness locking are employed. To avoid interpenetration a curvature based reversible adaptive meshing is devised. Kirchhoff condition is employed to reduce dynamic oscillation of new inserted nodes. The predicted drape shapes appear to be realistic with interpenetration avoided.


Fine tuned Predicted Summary Text : 
['A curvature based reversible adaptive meshing is devised. Kirchhoff condition is employed to reduce dynamic oscillation of new inserted nodes. The predicted drape shapes appear to be realistic with interpenetration avoided.']


Not Fine tuned Predicted Summary Text : 
solid shell elements attending shear trapezoidal and thickness locking are employed. to avoid interpenetration a curvature based reversible adaptive meshing is devised.


Original Text : 
A computationally efficient C0 finite element modeling in the framework of newly developed inverse hyperbolic shear deformation theory by the authors 

In [24]:
print('\n\nSummary Text: ')

# Replace the following string with your custom text
custom_text = "The first class to run will have a minimum of 5 students of the same age group. Please note that at this point we cannot guarantee class placement - joining the waitlist is only an expression of your interest, so we can determine how many teachers we need, and for which classes there is demand. We'll try to offer more options in the near future."
print('\n\nText: ')
print(custom_text)

print('\n\nFine tuned Predicted Summary Text : ')
print(model.predict(custom_text))
print("======================================================\n\n")

# Preprocess the custom text
preprocess_text = custom_text.strip().replace("\n", "")

# Create the T5 prepared text for summarization
t5_prepared_Text = "summarize: " + preprocess_text

# Tokenize the text
tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)

# Generate the summary
summary_ids = no_tune_model.generate(tokenized_text,
                                     num_beams=4,
                                     no_repeat_ngram_size=2,
                                     min_length=30,
                                     max_length=SUMMARY_LEN,
                                     early_stopping=True)

# Decode and print the generated summary
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(output)



Summary Text: 


Text: 
The first class to run will have a minimum of 5 students of the same age group. Please note that at this point we cannot guarantee class placement - joining the waitlist is only an expression of your interest, so we can determine how many teachers we need, and for which classes there is demand. We'll try to offer more options in the near future.


Fine tuned Predicted Summary Text : 
['We cannot guarantee class placement at this point. Joining the waitlist is only an expression of your interest. We can determine how many teachers we need, and for which classes there is demand.']


the first class to run will have a minimum of 5 students of the same age group. joining the waitlist is only an expression of interest, so we can determine how many teachers we need, and for which classes there is demand.


In [20]:
!pip install rouge-score

In [21]:
# Example usage
correct_predictions = 0

for index in range(len(df)):
    # Assuming model.predict() returns the predicted summary
    predicted_summary = model.predict(df['RHS'][index])

    # Comparing the predicted summary with the actual summary
    if predicted_summary == df['Abstract'][index]:
        correct_predictions += 1

# Calculate accuracy percentage
accuracy_percentage = (correct_predictions / len(df)) * 100

# Print the result
print(f'Accuracy: {accuracy_percentage:.2f}%')

Accuracy: 0.00%


In [22]:
!pip install sentence-transformers

In [25]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model
model_check = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Example usage
similarity_scores = []

for index in range(len(df)):
    # Assuming model.predict() returns the predicted summary
    predicted_summary_embedding = model_check.encode([df['RHS'][index]])[0]

    # Calculate semantic similarity
    similarity = util.pytorch_cos_sim(predicted_summary_embedding, model_check.encode([df['RHS'][index]]))[0][0].item()
    similarity_scores.append(similarity)

# Calculate average similarity score
average_similarity = sum(similarity_scores) / len(similarity_scores)

# Print the result
print(f'Average Semantic Similarity: {average_similarity:.2f}')

Average Semantic Similarity: 1.00
